In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import pickle


In [63]:
#aiodrive - CMU RI
import pandas as pd
# test = pd.read_pickle(r'C:/aiodrive/control/000000.pkl')
# print(test[0])
# print(len(test))
test = pd.read_pickle(r'C:/aiodrive/velocorr/000000.pkl')
print(test[0])
# print(test[10])
# dat = open(r'C:/aiodrive/lidar_velodyne/000001.bin').read_bytes()
# print(dat)
gps_i = np.loadtxt('C:/aiodrive/gps/000000.txt')
print(gps_i)
gps_i = np.loadtxt('C:/aiodrive/gps/000001.txt')
print(gps_i)
gps_i = np.loadtxt('C:/aiodrive/gps/000002.txt')
print(gps_i)

[[ 9.99723826e-01  1.41205925e-05  2.35004678e-02  2.34897002e-02]
 [ 4.53335213e-06  9.99999685e-01 -7.93715611e-04 -1.39734900e-03]
 [-2.35004716e-02  7.93602944e-04  9.99723511e-01 -1.18903333e-01]
 [-4.85722573e-17  4.16333634e-17  2.77555756e-17  1.00000000e+00]]
[-0.00013423 -0.00047382  0.04371029]
[-0.00013244 -0.00048092  0.04543356]
[-0.00011485 -0.00048143  0.04816017]


In [81]:
#SynLiDAR
import glob

def read_points(path):
    scan = np.fromfile(path, dtype=np.float32)
    scan = scan.reshape((-1, 4))  # [x,y,z,intensity]
    return scan

def read_label(path):
    label = np.fromfile(path, dtype=np.uint32)
    label = label.reshape((-1))
    return label

f_path = "C:/SynLiDAR/01/velodyne/000007.bin"

scan = read_points(f_path)
# print(np.shape(scan))

label_path = f_path.replace('velodyne', 'labels').replace('bin', 'label')
labels = read_label(label_path)

assert scan.shape[0] == labels.shape[0]

print(labels)


(97819,)


In [106]:
#CODD
import h5py

filename = 'C:/CODD/data/m1v7p7s769.hdf5'
vidx = 0

with h5py.File(filename, 'r') as hf:
#     pcls = hf['point_cloud'][:]
    #[frames, vehicles, points_per_cloud, 4]
    pcls = hf['point_cloud'][:, vidx ,: , :3]
    #[frames, points_per_cloud, rgb]
    
#     pose = hf['lidar_pose'][:]
    #[frames, vehicles, (x,y,z, rotx, roty, rotz)]
    pose = hf['lidar_pose'][:, vidx, :]
    
# print(np.shape(pcls))
# print(np.shape(pose))



[2.1274672e+02 1.9914983e+02 2.3481555e+00 4.6902906e-02 4.4329758e-03
 1.2788726e-06]
[2.1306000e+02 1.9914983e+02 2.3484421e+00 7.9844911e-03 4.4327090e-03
 1.6378874e-06]
